In [1]:
import cv2
import numpy as np
from ultralytics import YOLO
import time

/Users/evinjaff/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
def drawBackgroundText(image, x, y, text, foreground_color, size, thickness):
    (label_width, label_height), baseline = cv2.getTextSize(
        text, cv2.FONT_HERSHEY_SIMPLEX, size, thickness
    )

    cv2.rectangle(
        image,
        (x // 2 - label_width // 2, y),
        (x // 2 + label_width // 2, y - label_height),
        (0, 0, 0),
        -1,
    )

    cv2.putText(
        image,
        text,
        (x // 2 - label_width // 2, y),
        cv2.FONT_HERSHEY_SIMPLEX,
        size,
        foreground_color,
        thickness,
        cv2.LINE_AA,
    )

In [37]:
frame = cv2.imread(
    "/Users/evinjaff/github/oatmeal-cv-server/server/add_005.jpg"
)

w = int(frame.shape[1])
h = int(frame.shape[0])

model = YOLO("best.pt")

finalImg = frame

# get prediction
predictionObject = model(frame)

for allPreds in predictionObject:
    for pred in allPreds:
        if np.array(pred.boxes.conf)[0] < 0.7:
            pass

        # get label
        labelNum = np.array(pred.boxes.cls)[0]
        labelName = allPreds.names[labelNum]

        # change item foundDictionaries to true
        if labelName in allRequired.keys():
            allRequired[labelName] = True
        if labelName in allDistractors.keys():
            allDistractors[labelName] = True

        # Draw bounding box for object
        box = np.array(pred.boxes.xyxy).flatten().astype(int)
        # randomColor = tuple(np.random.random(size=3) * 256)
        randomColor = (0,255,0)
        cv2.rectangle(
            finalImg,
            (box[0], box[1]),
            (box[2], box[3]),
            randomColor,
            4,
        )
        (label_width, label_height), baseline = cv2.getTextSize(
            f"{labelName}: {round(float(np.array(pred.boxes.conf)[0]), 2)}",
            cv2.FONT_HERSHEY_SIMPLEX,
            1,
            2,
        )
        cv2.rectangle(
            finalImg,
            (box[0], box[1]),
            (box[0] + label_width, box[1] - label_height),
            randomColor,
            -1,
        )
        cv2.putText(
            finalImg,
            f"{labelName}: {round(float(np.array(pred.boxes.conf)[0]), 2)}",
            (box[0], box[1]),
            cv2.FONT_HERSHEY_SIMPLEX,
            1,
            (0, 0, 0),
            2,
            cv2.LINE_AA,
        )


cv2.imwrite('image.png', finalImg)


0: 384x640 1 tongs, 1 bowl, 1 pan, 1 pepper, 1 measuring_cup_glass, 1 small_spoon, 285.9ms
Speed: 1.9ms preprocess, 285.9ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)


True

: 

In [31]:
# Video version
# cap = cv2.VideoCapture(
#     "/Volumes/lconnor/Active/Patient_Recordings/50 Healthy Volunteers Video/AI007.mkv"
# )

cap = cv2.VideoCapture(0)

if cap.isOpened() == False:
    print("Error opening video file")

# cap = cv2.VideoCapture(
#     "/Volumes/lconnor-1/Active/Patient_Recordings/50 Healthy Volunteers Video/AI007.mkv"
# )

w = int(cap.get(3))
h = int(cap.get(4))
fps = int(cap.get(5))

model = YOLO("best.pt")


allItems = {
    "salt": False,
    "timer": False,
    "hot_pad": False,
    "oatmeal": False,
    "big_spoon": False,
    "measuring_spoons": False,
    "measuring_cup_1/2": False,
    "measuring_cup_coffee": False,
    "measuring_cup_full": False,
    "measuring_cup_1/3": False,
    "measuring_cup_1/4": False,
    "tongs": False,
    "scissors": False,
    "spatula": False,
    "bowl": False,
    "pan": False,
    "pepper": False,
    "measuring_cup_glass": False,
    "small_spoon": False,
    "measuring_cup_group": False,
    "glass": False,
}

allRequired = {
    "salt": False,
    "timer": False,
    "hot_pad": False,
    "oatmeal": False,
    "big_spoon": False,
    "measuring_spoons": False,
    "measuring_cup_1/2": False,
    "bowl": False,
    "pan": False,
    "measuring_cup_glass": False,
    "small_spoon": False,
}

allDistractors = {
    "measuring_cup_coffee": False,
    "measuring_cup_full": False,
    "measuring_cup_1/3": False,
    "measuring_cup_1/4": False,
    "tongs": False,
    "scissors": False,
    "spatula": False,
    "pepper": False,
    "measuring_cup_group": False,
    "glass": False,
}

index = 1
start_time = time.time()
while cap.isOpened():
    ret, frame = cap.read()
    finalImg = frame
    if ret == True:
        if index % 60 == 0:
            # extract only counter region
            detectionRegion = frame[(h - 2 * h // 3 + 20) : h, 0:w, :]
            regionH, regionW, regionC = detectionRegion.shape
            cv2.rectangle(frame, (0, h - regionH), (regionW, h), (0, 255, 0), 4)

            # get prediction
            predictionObject = model(detectionRegion)

            for allPreds in predictionObject:
                for pred in allPreds:
                    if np.array(pred.boxes.conf)[0] < 0.7:
                        pass

                    # get label
                    labelNum = np.array(pred.boxes.cls)[0]
                    labelName = allPreds.names[labelNum]

                    # change item foundDictionaries to true
                    if labelName in allRequired.keys():
                        # reset timer if new item added
                        if allRequired[labelName] == False:
                            start_time = time.time()
                        allRequired[labelName] = True
                    if labelName in allDistractors.keys():
                        allDistractors[labelName] = True

                    # Draw bounding box for object
                    box = np.array(pred.boxes.xyxy).flatten().astype(int)
                    randomColor = tuple(np.random.random(size=3) * 256)
                    cv2.rectangle(
                        finalImg,
                        (box[0], box[1] + h // 3 + 20),
                        (box[2], box[3] + h // 3 + 20),
                        randomColor,
                        4,
                    )
                    (label_width, label_height), baseline = cv2.getTextSize(
                        f"{labelName}: {round(float(np.array(pred.boxes.conf)[0]), 2)}",
                        cv2.FONT_HERSHEY_SIMPLEX,
                        1,
                        2,
                    )
                    cv2.rectangle(
                        finalImg,
                        (box[0], box[1] + h // 3 + 20),
                        (box[0] + label_width, box[1] + h // 3 + 20 - label_height),
                        randomColor,
                        -1,
                    )
                    cv2.putText(
                        finalImg,
                        f"{labelName}: {round(float(np.array(pred.boxes.conf)[0]), 2)}",
                        (box[0], box[1] + h // 3 + 20),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        1,
                        (0, 0, 0),
                        2,
                        cv2.LINE_AA,
                    )

                # ========= TEXT DISPLAY ========#
                # Required Items
                cv2.rectangle(finalImg, (0, 0), (540, h), (255, 255, 255), -1)
                cv2.putText(
                    finalImg,
                    "Required Items",
                    (0, 50),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    2,
                    (255, 0, 0),
                    4,
                    cv2.LINE_AA,
                )

                iter = 0
                for k, v in allRequired.items():
                    cv2.putText(
                        finalImg,
                        f"{k}: ",
                        (0, 50 + 40 + (30 * iter)),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        1,
                        (0, 0, 0),
                        2,
                        cv2.LINE_AA,
                    )

                    (label_width, label_height), baseline = cv2.getTextSize(
                        f"{k}: ", cv2.FONT_HERSHEY_SIMPLEX, 1, 2
                    )

                    if v == True:
                        cv2.putText(
                            finalImg,
                            "PRESENT",
                            (0 + label_width, 50 + 40 + (30 * iter)),
                            cv2.FONT_HERSHEY_SIMPLEX,
                            1,
                            (0, 180, 0),
                            2,
                            cv2.LINE_AA,
                        )
                    else:
                        cv2.putText(
                            finalImg,
                            "ABSENT",
                            (0 + label_width, 50 + 40 + (30 * iter)),
                            cv2.FONT_HERSHEY_SIMPLEX,
                            1,
                            (0, 0, 255),
                            2,
                            cv2.LINE_AA,
                        )
                    iter += 1

                # Distractor Items
                cv2.putText(
                    finalImg,
                    "Distractor Items",
                    (0, h - h // 2),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    2,
                    (255, 0, 0),
                    4,
                    cv2.LINE_AA,
                )

                iter = 0
                for k, v in allDistractors.items():
                    cv2.putText(
                        finalImg,
                        f"{k}: ",
                        (0, h - h // 2 + 40 + (30 * iter)),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        1,
                        (0, 0, 0),
                        2,
                        cv2.LINE_AA,
                    )

                    (label_width, label_height), baseline = cv2.getTextSize(
                        f"{k}: ", cv2.FONT_HERSHEY_SIMPLEX, 1, 2
                    )

                    if v == True:
                        cv2.putText(
                            finalImg,
                            "PRESENT",
                            (0 + label_width, h - h // 2 + 40 + (30 * iter)),
                            cv2.FONT_HERSHEY_SIMPLEX,
                            1,
                            (0, 180, 0),
                            2,
                            cv2.LINE_AA,
                        )
                    else:
                        cv2.putText(
                            finalImg,
                            "ABSENT",
                            (0 + label_width, h - h // 2 + 40 + (30 * iter)),
                            cv2.FONT_HERSHEY_SIMPLEX,
                            1,
                            (0, 0, 255),
                            2,
                            cv2.LINE_AA,
                        )
                    iter += 1

                # ITEM LIST STATE CHECKS
                if all(value for value in allRequired.values()):
                    # all required and no distractors
                    if not any(value for value in allDistractors.values()):
                        drawBackgroundText(
                            finalImg,
                            w,
                            50,
                            "MOVING TO ACTION DETECTION",
                            (255, 0, 0),
                            2,
                            4,
                        )
                    # all required and some distractors
                    else:
                        drawBackgroundText(
                            finalImg,
                            w,
                            50,
                            "Anything you don't need?",
                            (255, 0, 0),
                            2,
                            4,
                        )
                else:
                    elapsed_time = start_time - time.time()
                    if elapsed_time >= 8:
                        drawBackgroundText(
                            finalImg,
                            w,
                            50,
                            "Are you missing something?",
                            (255, 0, 0),
                            2,
                            4,
                        )
                        start_time = time.time()

                    else:
                        drawBackgroundText(
                            finalImg,
                            w,
                            50,
                            "PARTICIPANT STILL PICKING",
                            (255, 0, 0),
                            2,
                            4,
                        )

                # OUTPUT FRAME
                cv2.imshow("OUTPUT", finalImg)

        if cv2.waitKey(1) & 0xFF == ord("q"):
            break

        index += 1

    else:
        break


cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)


0: 256x640 (no detections), 218.3ms
Speed: 1.6ms preprocess, 218.3ms inference, 0.4ms postprocess per image at shape (1, 3, 256, 640)

0: 256x640 (no detections), 200.3ms
Speed: 1.3ms preprocess, 200.3ms inference, 0.3ms postprocess per image at shape (1, 3, 256, 640)

0: 256x640 (no detections), 213.3ms
Speed: 1.2ms preprocess, 213.3ms inference, 0.3ms postprocess per image at shape (1, 3, 256, 640)

0: 256x640 (no detections), 235.5ms
Speed: 1.9ms preprocess, 235.5ms inference, 0.3ms postprocess per image at shape (1, 3, 256, 640)

0: 256x640 (no detections), 235.8ms
Speed: 1.4ms preprocess, 235.8ms inference, 0.3ms postprocess per image at shape (1, 3, 256, 640)

0: 256x640 (no detections), 246.3ms
Speed: 2.2ms preprocess, 246.3ms inference, 0.3ms postprocess per image at shape (1, 3, 256, 640)

0: 256x640 (no detections), 236.0ms
Speed: 1.7ms preprocess, 236.0ms inference, 0.5ms postprocess per image at shape (1, 3, 256, 640)

0: 256x640 (no detections), 237.4ms
Speed: 2.5ms prepr

KeyboardInterrupt: 